In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [2]:
ex1 = pd.read_parquet('train/2.신용정보/2018_신용정보_전체.parquet')

In [3]:
member_df = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')
member_seg = member_df[['ID', 'Segment']].drop_duplicates(subset='ID')
ex1 = ex1.merge(member_seg, on='ID', how='left')



In [4]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 'RV전환가능여부', '일시불ONLY전환가능여부', '카드이용한도금액_B1M', '카드이용한도금액_B2M', '특별한도보유여부_R3M', '연체감액여부_R3M', '한도심사요청건수', '한도요청거절건수', '한도심사요청후경과월', '한도심사거절후경과월', '시장단기연체여부_R6M', '시장단기연체여부_R3M', '시장연체상환여부_R6M', '시장연체상환여부_R3M', 'rv최초시작후경과일', 'Segment']

문자형 컬럼 이름:
['ID', '자발한도감액횟수_R12M', '한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부', '한도심사요청건수', 'Segment']

숫자형 컬럼 이름:
['기준년월', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '월

In [5]:
cols = ['자발한도감액횟수_R12M', '한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부', '한도심사요청건수']
print(ex1[cols].head())

  자발한도감액횟수_R12M 한도증액횟수_R12M 카드론동의여부 RV전환가능여부 한도심사요청건수
0            0회          0회       Y        N       0회
1            0회          0회       Y        Z       0회
2            0회          0회       Y        N       0회
3            0회          0회       Y        N       0회
4            0회          0회       Y        Z       0회


In [6]:
count_cols2 = [
    '자발한도감액횟수_R12M',
    '한도증액횟수_R12M',
    '한도심사요청건수'
]

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex1[col] = ex1[col].astype(str).str.replace('회', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [7]:
le_cols = ['카드론동의여부', 'RV전환가능여부']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes


In [8]:
ex1.head()

,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일,Segment
0,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0,0,3,3,0,0,0,0,99999999,D
1,201807,TRAIN_000001,0,9996,5718,41996,90611,14.793821,14.834873,10.622446,...,0,0,3,3,0,0,0,0,322,E
2,201807,TRAIN_000002,0,88193,35207,0,0,22.014276,17.875321,17.155829,...,0,0,3,3,0,0,0,0,2378,C
3,201807,TRAIN_000003,0,19062,6531,0,0,22.998014,22.999453,19.293674,...,0,0,3,3,0,0,0,0,99999999,D
4,201807,TRAIN_000004,0,177222,47149,48000,155020,14.661948,10.897410,10.654587,...,0,0,3,3,0,0,0,0,99999999,E


In [9]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 'RV전환가능여부', '일시불ONLY전환가능여부', '카드이용한도금액_B1M', '카드이용한도금액_B2M', '특별한도보유여부_R3M', '연체감액여부_R3M', '한도심사요청건수', '한도요청거절건수', '한도심사요청후경과월', '한도심사거절후경과월', '시장단기연체여부_R6M', '시장단기연체여부_R3M', '시장연체상환여부_R6M', '시장연체상환여부_R3M', 'rv최초시작후경과일', 'Segment']

문자형 컬럼 이름:
['ID', 'Segment']

숫자형 컬럼 이름:
['기준년월', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 'RV전환가능여부'

In [10]:
ex1 = ex1.fillna(-1)
ex1

,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일,Segment
0,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0,0,3,3,0,0,0,0,99999999,D
1,201807,TRAIN_000001,0,9996,5718,41996,90611,14.793821,14.834873,10.622446,...,0,0,3,3,0,0,0,0,322,E
2,201807,TRAIN_000002,0,88193,35207,0,0,22.014276,17.875321,17.155829,...,0,0,3,3,0,0,0,0,2378,C
3,201807,TRAIN_000003,0,19062,6531,0,0,22.998014,22.999453,19.293674,...,0,0,3,3,0,0,0,0,99999999,D
4,201807,TRAIN_000004,0,177222,47149,48000,155020,14.661948,10.897410,10.654587,...,0,0,3,3,0,0,0,0,99999999,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,20070,10167,0,54445,15.243670,11.900516,10.545876,...,0,0,3,3,0,0,0,0,99999999,E
2399996,201812,TRAIN_399996,0,84217,31159,0,156800,14.843464,15.652063,11.102726,...,0,0,3,3,0,0,0,0,99999999,D
2399997,201812,TRAIN_399997,0,52612,19429,0,0,17.038599,17.966213,13.278475,...,0,0,3,3,0,0,0,0,99999999,C
2399998,201812,TRAIN_399998,0,10002,4228,90004,180906,15.182880,11.901089,10.594124,...,0,0,3,3,0,0,0,0,99999999,E


In [11]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.01 ].index.tolist()
low_var

['자발한도감액횟수_R12M',
 '자발한도감액금액_R12M',
 '특별한도보유여부_R3M',
 '연체감액여부_R3M',
 '한도심사요청건수',
 '한도요청거절건수',
 '한도심사요청후경과월',
 '한도심사거절후경과월',
 '시장단기연체여부_R6M',
 '시장단기연체여부_R3M',
 '시장연체상환여부_R6M',
 '시장연체상환여부_R3M']

In [12]:
cols_to_drop = ['자발한도감액횟수_R12M',
 '자발한도감액금액_R12M',
 '특별한도보유여부_R3M',
 '연체감액여부_R3M',
 '한도심사요청건수',
 '한도요청거절건수',
 '한도심사요청후경과월',
 '한도심사거절후경과월',
 '시장단기연체여부_R6M',
 '시장단기연체여부_R3M',
 '시장연체상환여부_R6M',
 '시장연체상환여부_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [13]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): ['강제한도감액후경과월', '자발한도감액후경과월', '한도증액금액_R12M', '일시불ONLY전환가능여부', '한도증액횟수_R12M', '최초한도금액', 'RV약정청구율', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '카드론동의여부', 'rv최초시작후경과일', 'RV신청일자', 'RV전환가능여부', '한도증액후경과월', '월상환론상향가능한도금액']


In [14]:
import pandas as pd
import numpy as np

num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)


high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 14


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,카드이용한도금액_B1M,카드이용한도금액_B2M,0.984564,0.303262,0.303258
1,카드이용한도금액,카드이용한도금액_B1M,0.977891,0.304435,0.303262
2,카드이용한도금액,카드이용한도금액_B2M,0.967891,0.304435,0.303258
3,상향가능한도금액,상향가능CA한도금액,0.963278,0.050764,0.064575
4,카드이용한도금액,CA한도금액,0.924623,0.304435,0.292502
5,CA이자율_할인전,RV현금서비스이자율_할인전,0.923218,0.118189,0.096870
6,CA한도금액,카드이용한도금액_B1M,0.918732,0.292502,0.303262
7,CA이자율_할인전,RV일시불이자율_할인전,0.917201,0.118189,0.073838
8,RV일시불이자율_할인전,RV현금서비스이자율_할인전,0.913497,0.073838,0.096870
9,CA한도금액,카드이용한도금액_B2M,0.911089,0.292502,0.303258


In [15]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))


print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 8
제거할 피처 목록:
['CL이자율_할인전', '카드이용한도금액_B2M', '상향가능한도금액', '카드이용한도금액_B1M', 'RV현금서비스이자율_할인전', 'CA한도금액', 'RV일시불이자율_할인전', 'RV최소결제비율']


In [16]:
cols_to_drop = ['CL이자율_할인전', '카드이용한도금액_B2M', '상향가능한도금액', '카드이용한도금액_B1M', 'RV현금서비스이자율_할인전', 'CA한도금액', 'RV일시불이자율_할인전', 'RV최소결제비율']
ex1.drop(columns=cols_to_drop, inplace=True)

In [17]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [18]:
ex1

,기준년월,ID,카드이용한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,상향가능CA한도금액,Segment,Segment_encoded
0,201807,TRAIN_000000,19354,0,0,22.995207,0,D,3
1,201807,TRAIN_000001,9996,41996,90611,14.793821,1,E,4
2,201807,TRAIN_000002,88193,0,0,22.014276,0,C,2
3,201807,TRAIN_000003,19062,0,0,22.998014,0,D,3
4,201807,TRAIN_000004,177222,48000,155020,14.661948,0,E,4
...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,20070,0,54445,15.243670,1,E,4
2399996,201812,TRAIN_399996,84217,0,156800,14.843464,3,D,3
2399997,201812,TRAIN_399997,52612,0,0,17.038599,0,C,2
2399998,201812,TRAIN_399998,10002,90004,180906,15.182880,1,E,4


In [19]:
from sklearn.preprocessing import LabelEncoder

df_corr = ex1.drop(columns=['기준년월', 'ID']).copy()

le = LabelEncoder()
df_corr['Segment_encoded'] = le.fit_transform(df_corr['Segment'])

correlations = df_corr.corr(numeric_only=True)['Segment_encoded'].drop('Segment_encoded')

selected_cols = correlations[correlations.abs() > 0.1].index.tolist()

final_cols = ['기준년월', 'ID','Segment'] + selected_cols

ex1_filtered = ex1[final_cols]


In [20]:
cols = ex1_filtered.columns.tolist()
cols

['기준년월', 'ID', 'Segment', '카드이용한도금액', 'CA이자율_할인전']

In [21]:
ex1_filtered.to_parquet('신용_전처리_Segment.parquet', index=False)